In [2]:
from numerals.grammar import Number, Digit, Phrase, base_numerals_grammar, english_numerals_grammar
from numerals.meaning import universe as numerals_universe

from ultk.language.grammar import Grammar, Rule, GrammaticalExpression, Meaning

from copy import deepcopy

In [3]:
print(base_numerals_grammar)

Rules:
	~Number -> add(~Phrase, ~Number)
	~Number ->  (~Phrase)
	~Number -> sub(~Phrase, ~Number)
	<class 'bool'> -> *(~Number, tuple[ultk.language.semantics.Referent])
	tuple[ultk.language.semantics.Referent] -> .


In [4]:
def get_expressions(g: Grammar) -> list[str]:
    expressions_by_meaning: dict[
        Meaning, GrammaticalExpression
    ] = g.get_unique_expressions(
        4,
        max_size=2 ** len(numerals_universe),
        unique_key=lambda expr: expr.evaluate(numerals_universe),
        compare_func=lambda e1, e2: len(e1) < len(e2),
    )
    return {e.term_expression: {m.name for m in e.meaning if e.meaning[m]} for e in expressions_by_meaning.values()}

In [5]:
# test adding a rule

def add_digit_rule(g: Grammar, num: int) -> Grammar:
    g.add_rule(
        Rule(
            name=f"_{num}",
            lhs=Number, 
            rhs=None, # formally, of type Digit, for None since terminal
            func=lambda _: num,
        )
    )
    return g

def add_mult_rule(g: Grammar, num: int) -> Grammar:
    g.add_rule(
        Rule(
            name=f"x{num}",
            lhs=Phrase,
            rhs=[Number],
            func=lambda x: x * num,
        )
    )
    return g


In [6]:
updated_grammar = add_digit_rule(deepcopy(base_numerals_grammar), 1)
updated_grammar = add_mult_rule(updated_grammar, 10)
print(updated_grammar)

Rules:
	~Number -> add(~Phrase, ~Number)
	~Number ->  (~Phrase)
	~Number -> sub(~Phrase, ~Number)
	~Number -> _1
	<class 'bool'> -> *(~Number, tuple[ultk.language.semantics.Referent])
	tuple[ultk.language.semantics.Referent] -> .
	~Phrase -> x10(~Number)


In [7]:
print(base_numerals_grammar)

Rules:
	~Number -> add(~Phrase, ~Number)
	~Number ->  (~Phrase)
	~Number -> sub(~Phrase, ~Number)
	<class 'bool'> -> *(~Number, tuple[ultk.language.semantics.Referent])
	tuple[ultk.language.semantics.Referent] -> .


In [8]:
get_expressions(base_numerals_grammar)

{}

In [9]:
get_expressions(updated_grammar)

{'*(_1, .)': {1},
 '*(add(x10(_1), _1), .)': {11},
 '*( (x10(_1)), .)': {10},
 '*(sub(x10(_1), _1), .)': {9}}

In [10]:
print(english_numerals_grammar)

Rules:
	~Number -> add(~Phrase, ~Number)
	~Number -> eight
	~Number -> five
	~Number -> four
	~Number -> nine
	~Number -> one
	~Number ->  (~Phrase)
	~Number -> seven
	~Number -> six
	~Number -> ten
	~Number -> three
	~Number -> two
	<class 'bool'> -> *(~Number, tuple[ultk.language.semantics.Referent])
	tuple[ultk.language.semantics.Referent] -> .
	~Phrase -> x10(~Number)


In [11]:
get_expressions(english_numerals_grammar)

{'*(eight, .)': {8},
 '*(five, .)': {5},
 '*(four, .)': {4},
 '*(nine, .)': {9},
 '*(one, .)': {1},
 '*(seven, .)': {7},
 '*(six, .)': {6},
 '*(ten, .)': {10},
 '*(three, .)': {3},
 '*(two, .)': {2},
 '*(add(x10(eight), eight), .)': {88},
 '*(add(x10(eight), five), .)': {85},
 '*(add(x10(eight), four), .)': {84},
 '*(add(x10(eight), nine), .)': {89},
 '*(add(x10(eight), one), .)': {81},
 '*(add(x10(eight), seven), .)': {87},
 '*(add(x10(eight), six), .)': {86},
 '*( (x10(nine)), .)': {90},
 '*(add(x10(eight), three), .)': {83},
 '*(add(x10(eight), two), .)': {82},
 '*(add(x10(five), eight), .)': {58},
 '*(add(x10(five), five), .)': {55},
 '*(add(x10(five), four), .)': {54},
 '*(add(x10(five), nine), .)': {59},
 '*(add(x10(five), one), .)': {51},
 '*(add(x10(five), seven), .)': {57},
 '*(add(x10(five), six), .)': {56},
 '*( (x10(six)), .)': {60},
 '*(add(x10(five), three), .)': {53},
 '*(add(x10(five), two), .)': {52},
 '*(add(x10(four), eight), .)': {48},
 '*(add(x10(four), five), .)':

next, try removing a grammar rule.

In [ ]:
# iterate over rules and find one corresponding to an M rule or D rule
# then del from dict
updated_grammar._rules

[Rule(name='add', lhs=~Number, rhs=(~Phrase, ~Number), func=<function add at 0x123c6b100>, weight=1.0),
 Rule(name=' ', lhs=~Number, rhs=(~Phrase,), func=<function phrase at 0x123c6b240>, weight=1.0),
 Rule(name='sub', lhs=~Number, rhs=(~Phrase, ~Number), func=<function sub at 0x123c6b1a0>, weight=1.0),
 Rule(name='_1', lhs=~Number, rhs=None, func=<function add_digit_rule.<locals>.<lambda> at 0x123c80e00>, weight=1.0),
 Rule(name='*', lhs=<class 'bool'>, rhs=(~Number, tuple[ultk.language.semantics.Referent]), func=<function apply_et at 0x123c6b060>, weight=1.0),
 Rule(name='.', lhs=tuple[ultk.language.semantics.Referent], rhs=None, func=<function bind at 0x123c6afc0>, weight=1.0),
 Rule(name='x10', lhs=~Phrase, rhs=[~Number], func=<function add_mult_rule.<locals>.<lambda> at 0x123c80f40>, weight=1.0)]

In [15]:
def is_D_rule(rule: Rule) -> bool:
    return rule.lhs == Number and rule.rhs is None

for k, v in updated_grammar._rules_by_name.items():
    print(k, is_D_rule(v))

add False
* False
. False
  False
sub False
_1 True
x10 False


In [16]:
def is_M_rule(rule: Rule) -> bool:
    return rule.lhs == Phrase and rule.rhs == [Number]

for k, v in updated_grammar._rules_by_name.items():
    print(k, is_M_rule(v))

add False
* False
. False
  False
sub False
_1 False
x10 True


In [ ]:
import random

def remove_D_rule(grammar: Grammar) -> Grammar:
    D_rules = [rule for rule in grammar._rules_by_name if is_D_rule(grammar._rules_by_name[rule])]
    rule_to_remove = random.choice(D_rules)
    new_rules = [rule for name, rule in grammar._rules_by_name.items() if name != rule_to_remove]
    new_grammar = Grammar(grammar._start)
    for rule in new_rules:
        new_grammar.add_rule(rule)
    return new_grammar

In [18]:
updated_grammar = remove_D_rule(updated_grammar)
print(updated_grammar)

TypeError: unhashable type: 'list'